In [50]:
# Experiments on entire text instead of just snippets

import pandas as pd
import glob
import numpy as np 
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import learning_curve
from sklearn.model_selection import ShuffleSplit
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier



In [2]:
# Get data 
species = glob.glob('*tsv')
np_array_list = []
for file_ in species:
    df = pd.read_csv(file_,index_col=None, header=0, sep="\t")
    np_array_list.append(df.as_matrix())

comb_np_array = np.vstack(np_array_list)
big_frame = pd.DataFrame(comb_np_array)


In [55]:
pd.set_option('display.max_colwidth', 1)
temp = big_frame[9].str.extract(r'<body>(.*)</body>', expand=False)

temp  = temp.str.replace('<b style="color:red; font-size:14pt"><u><i>(.*)</i></u></b>', '=ANIMAL=')
temp  = temp.str.replace('<title>', '')
temp  = temp.str.replace('</title>', '')
temp  = temp.str.replace('<p>', '')
temp  = temp.str.replace('</p>', '')
temp  = temp.str.replace('<title xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance">' ,'')
temp  = temp.str.replace('<p xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance">' ,'')

big_frame['text_clean'] = temp 


big_frame.head()

0                                    1     2  \
0  01_Natural_history  KBDDD02:000200891:mpeg21:a0057:ocr   1927   
1  05_Pest_control     KBNRC01:000057072:mpeg21:a0081:ocr   1939   
2  06_Hunt_economical  MMCODA01:000156057:mpeg21:a0005:ocr  1931   
3  05_Pest_control     MMCODA01:000158525:mpeg21:a0062:ocr  1937   
4  05_Pest_control     MMDA03:000111229:mpeg21:a0042:ocr    1915   

                                        3  \
0  Voorwaarts                               
1  Algemeen Handelsblad                     
2  Nieuwe Apeldoornsche courant             
3  Nieuwe Apeldoornsche courant             
4  Provinciale Drentsche en Asser courant   

                                                                                                                            4  \
0  Dieren die geofferd worden De mol en de mode.                                                                                
1  LAND VAN BEPROEVING door Edison Marshall                                                                                     
2  Het moeras, dat zich niet temmen Iaat. Alle pogingen om het productief te maken falen. De natuur wint het van de techniek.   
3  GEMENGD BUITENLAND „Ook een schoolstrijd&quot;. Boer in conflict met de leerplichtwet.                                       
4  GEMENGD NIEUWS.                                                                                                              

         5         6   7  \
0  artikel  0.933333  nl   
1  artikel  0.980097  nl   
2  artikel  0.964935  nl   
3  artikel  0.926916  nl   
4  artikel  0.872581  nl   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      8  \
0  deze zijde ven het himalaya-gebergte voorkomt er bestaat ook een watermol die in de natuurlijke historie bekend is als de =ANIMAL= dezen naam dankt het dier hieraan dat het een puntige snuit heeft en naar muskus ruik het leeft voornamelijk in</li><li>muskus ruik het leeft voornamelijk in rusland aan de oevers van den wolga het schijnt dat de visschen die dezen =ANIMAL= als hun voedsel gebruiken omdat hun vleesch dermate door dezen muskusgeur doortrokken is en zijn nabijheid in de buurt van</li><li>van vijvers brengt dan ook den schrik onder de visschers men gebruikt voor bont heel veel de buikzijde van dezen =ANIMAL= die een prachtig zilverwitten tint heeft dit bont is dan ook zeer kostbaar voor een mantel heeft men een honderdtal   
1  verder tijdens dien eersten langen dagmarsch leverden verscheidene andere klemmen eveneens resultaat op de klem op den dam bevatte een =ANIMAL= en in de vallen langs de kreek vond hij nog een paar van deze kleine roofdieren weliswaar waren hun huiden                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             
2  van het moeras naar de huizen der boeren terugkeeren andere dieren er zijn ook otters wezels waschberen opossums bunzings en <b>muskusratten</b> in het 

In [38]:
#Try the NaiveBayes classifier on the complete dataset 
labels = big_frame[0]
text = big_frame['text_clean']
print("length total: ", len(labels))
labels_train, labels_test, text_train, text_test = train_test_split(labels, text, test_size=0.3, random_state=1)
print("length train labels: ", len(labels_train))
print("length test labels: ", len(labels_test))


text_clf = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()), ('clf', MultinomialNB()),])
text_clf.fit(text_train, labels_train)
docs_test = text_test
predicted = text_clf.predict(docs_test)
print(metrics.classification_report(labels_test, predicted))


length total:  9931
length train labels:  6951
length test labels:  2980
                        precision    recall  f1-score   support

    01_Natural_history       1.00      0.00      0.01       333
  02_Nuisance_material       0.00      0.00      0.00       115
03_Nuisance_immaterial       0.00      0.00      0.00        41
       05_Pest_control       0.00      0.00      0.00       202
    06_Hunt_economical       0.98      0.35      0.52       578
          08_Accidents       0.00      0.00      0.00        83
         09_Figurative       0.00      0.00      0.00        93
        10_Other_beast       0.00      0.00      0.00         5
            11_Nobeast       0.52      1.00      0.69      1449
         12_Bagger_OCR       0.00      0.00      0.00        81

           avg / total       0.56      0.56      0.44      2980



/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [45]:
#text_clf = Pipeline([('tfidf', TfidfVectorizer(min_df=5, max_df=0.7, sublinear_tf=True, ngram_range=(1,2), analyzer='word', max_features=10000)), ('clf-svm', LinearSVC(class_weight='balanced', C=1.0, verbose=True))])
#text_clf = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()), ('clf', MultinomialNB()),])
text_clf = Pipeline([('tfidf', TfidfVectorizer(min_df=5, max_df=0.7, sublinear_tf=True, ngram_range=(1,2), analyzer='word', max_features=10000)), ('clf', SGDClassifier(loss='hinge', penalty='l2', alpha=1e-3, random_state=42, max_iter=5, tol=None)),])
#text_clf = Pipeline([('tfidf', TfidfVectorizer(min_df=5, max_df=0.7, sublinear_tf=True, ngram_range=(1,2), analyzer='word', max_features=10000)), ('clf', KNeighborsClassifier(n_neighbors=3, weights='uniform', algorithm='brute', leaf_size=30, p=2, metric='euclidean', metric_params=None, n_jobs=1))])
#text_clf = Pipeline([('tfidf', TfidfVectorizer(min_df=5, max_df=0.7, sublinear_tf=True, ngram_range=(1,2), analyzer='word', max_features=10000)), ('clf', DecisionTreeClassifier())])


from sklearn import metrics
from sklearn.model_selection import StratifiedKFold
import numpy as np

def customCrossValidation(X, y):
    ''' Perform a cross validation and print out the metrics '''
    skf = StratifiedKFold(n_splits=10)
    cm = None
    y_predicted_overall = None
    y_test_overall = None
    for train_index, test_index in skf.split(X, y):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        text_clf.fit(X_train, y_train)
        y_predicted = text_clf.predict(X_test)
        # collect the y_predicted per fold
        if y_predicted_overall is None:
            y_predicted_overall = y_predicted
            y_test_overall = y_test
        else: 
            y_predicted_overall = np.concatenate([y_predicted_overall, y_predicted])
            y_test_overall = np.concatenate([y_test_overall, y_test])
        cv_cm = metrics.confusion_matrix(y_test, y_predicted)
        # sum the cv per fold
        if cm is None:
            cm = cv_cm
        else:
            cm += cv_cm
    print (metrics.classification_report(y_test_overall, y_predicted_overall, digits=3))
    print (cm)
    
customCrossValidation(big_frame['text_clean'], big_frame[0])  



                        precision    recall  f1-score   support

    01_Natural_history      0.651     0.720     0.684      1144
  02_Nuisance_material      0.528     0.314     0.394       388
03_Nuisance_immaterial      0.533     0.073     0.128       110
       05_Pest_control      0.556     0.133     0.215       669
    06_Hunt_economical      0.805     0.726     0.763      1947
          08_Accidents      0.692     0.313     0.431       230
         09_Figurative      0.519     0.120     0.195       334
        10_Other_beast      0.000     0.000     0.000        22
            11_Nobeast      0.740     0.965     0.837      4848
         12_Bagger_OCR      0.000     0.000     0.000       239

           avg / total      0.692     0.730     0.685      9931

[[ 824    6    0    8   47    2    7    0  250    0]
 [  48  122    1   33   36    6    1    0  141    0]
 [  24    9    8    9   11    5    0    0   44    0]
 [ 125   61    0   89   90   10    5    0  287    2]
 [  81   16    0 

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [48]:
text_clf = Pipeline([('tfidf', TfidfVectorizer(min_df=5, max_df=0.7, sublinear_tf=True, ngram_range=(1,2), analyzer='word', max_features=10000)), ('clf', KNeighborsClassifier(n_neighbors=3, weights='uniform', algorithm='brute', leaf_size=30, p=2, metric='euclidean', metric_params=None, n_jobs=1))])
customCrossValidation(big_frame['text_clean'], big_frame[0])  



                        precision    recall  f1-score   support

    01_Natural_history      0.547     0.626     0.584      1144
  02_Nuisance_material      0.131     0.776     0.224       388
03_Nuisance_immaterial      0.323     0.091     0.142       110
       05_Pest_control      0.367     0.172     0.234       669
    06_Hunt_economical      0.863     0.601     0.708      1947
          08_Accidents      0.669     0.396     0.497       230
         09_Figurative      0.574     0.338     0.426       334
        10_Other_beast      0.000     0.000     0.000        22
            11_Nobeast      0.941     0.814     0.873      4848
         12_Bagger_OCR      0.390     0.163     0.230       239

           avg / total      0.769     0.655     0.690      9931

[[ 716  284    2   38   26    5   19    0   52    2]
 [  34  301    1   33    4    7    3    0    5    0]
 [  20   63   10    5    1    7    1    0    3    0]
 [ 142  323    1  115   40   18    5    0   25    0]
 [ 129  457    5 

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [51]:
text_clf = Pipeline([('tfidf', TfidfVectorizer(min_df=5, max_df=0.7, sublinear_tf=True, ngram_range=(1,2), analyzer='word', max_features=10000)), ('clf', DecisionTreeClassifier())])
customCrossValidation(big_frame['text_clean'], big_frame[0])  


                        precision    recall  f1-score   support

    01_Natural_history      0.510     0.525     0.517      1144
  02_Nuisance_material      0.337     0.369     0.352       388
03_Nuisance_immaterial      0.184     0.173     0.178       110
       05_Pest_control      0.248     0.251     0.249       669
    06_Hunt_economical      0.667     0.639     0.653      1947
          08_Accidents      0.366     0.357     0.361       230
         09_Figurative      0.345     0.359     0.352       334
        10_Other_beast      0.000     0.000     0.000        22
            11_Nobeast      0.825     0.825     0.825      4848
         12_Bagger_OCR      0.139     0.142     0.140       239

           avg / total      0.648     0.646     0.647      9931

[[ 601   44   15  112  104   19   61    6  165   17]
 [  42  143   14   71   32   21    8    0   50    7]
 [  15   16   19   19    9   10   10    0    9    3]
 [ 115   88   22  168   89   31   21    2  121   12]
 [ 138   42    8 

,1,2,3,4,5,6,7,8,9,10,11,12,13,text_clean
0,,,,,,,,,,,,,,
01_Natural_history,1144,1144,1144,1144,1144,1144,1144,1144,1144,1144,1144,1144,1144,1144
02_Nuisance_material,388,388,388,388,388,388,388,388,388,388,388,388,388,388
03_Nuisance_immaterial,110,110,110,110,110,110,110,110,110,110,110,110,110,110
05_Pest_control,669,669,669,669,669,669,669,669,669,669,669,669,669,669
06_Hunt_economical,1947,1947,1947,1947,1947,1947,1947,1947,1947,1947,1947,1947,1947,1947
08_Accidents,230,230,230,230,230,230,230,230,230,230,230,230,230,230
09_Figurative,334,334,334,334,334,334,334,334,334,334,334,334,334,334
10_Other_beast,22,22,22,22,22,22,22,22,22,22,22,22,22,22
11_Nobeast,4848,4848,4848,4847,4848,4848,4848,4848,4848,4848,4848,4848,4848,4848
